In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from timeit import default_timer as timer  
from numba import jit, cuda

In [2]:
df = pd.read_csv(r'us_accidents_weather_data.csv')
#data = df[['ID', 'Severity', 'Weather_Timestamp', 'Temperature(F)','Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)','Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition','Sunrise_Sunset']]
data = df[['Severity', 'Temperature(F)','Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)','Wind_Speed(mph)', 'Precipitation(in)']]
data = data.dropna()
data = data[data['Severity'] == 4]
data = data.reset_index(drop=True)
#data = data.head(150000)
print(len(data))
print(data.head(5))
k = 3
eps = 5
iterationsMax = 20
clusters = pd.DataFrame(data = 0.0, columns=data.columns, index=range(k))
c = np.full(len(data), -1)

62669
   Severity  Temperature(F)  Wind_Chill(F)  Humidity(%)  Pressure(in)  \
0         4            19.9            7.3         81.0         29.85   
1         4            19.9            7.3         81.0         29.85   
2         4            15.8            2.7         73.0         29.87   
3         4            17.1            0.6         77.0         29.91   
4         4            17.6            9.3         86.0         29.94   

   Visibility(mi)  Wind_Speed(mph)  Precipitation(in)  
0             2.0             12.7               0.00  
1             2.0             12.7               0.00  
2             4.0             11.5               0.00  
3             2.5             19.6               0.01  
4             9.0              5.8               0.00  


In [3]:
# data: points df
# clusters: clusters df
@jit(target_backend='cuda', nogil=True, cache=True) 
def initClusters(data, clusters):
    for i, ii in enumerate(clusters.iloc[0]):
        clmax = data.max().iloc[i]
        clmin = data.min().iloc[i]
        for j,jj in clusters.iterrows():
            clusters.iloc[j].iloc[i] = (clmax - clmin) * np.random.random() + clmin

initClusters(data, clusters)
print(clusters)

C:\Users\justi\AppData\Local\Temp\ipykernel_23148\3457171471.py:3: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda', nogil=True, cache=True)
C:\Users\justi\AppData\Local\Temp\ipykernel_23148\3457171471.py:3: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "initClusters" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_23148\3457171471.py (3)

File "..\..\..\..\AppData\Local\Temp\ipykernel_23148\3457171471.py", line 3:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda', nogil=True, cache=True)
C:\Users\jus

   Severity  Temperature(F)  Wind_Chill(F)  Humidity(%)  Pressure(in)  \
0       4.0       45.280928     -18.049335    26.531568     28.272819   
1       4.0       19.632528     114.092487    73.229098     28.589851   
2       4.0       13.549517     -34.412489     9.207838     29.914190   

   Visibility(mi)  Wind_Speed(mph)  Precipitation(in)  
0       33.097954       211.783523           3.097339  
1        1.664061       149.680531           1.437676  
2       76.809146       213.445194           1.148373  


In [4]:
# p: index of point 
# c: index of cluster
# clusters: clusters df
# data: points df
@jit(target_backend='cuda', nogil=True, cache=True) 
def sse(p: int, c: int, clusters: pd.DataFrame, data: pd.DataFrame) -> float:
    error: float = 0.0
    for i, ii in enumerate(clusters):
        error += (clusters.iloc[c][i] - data.iloc[p][i])**2
    return error


sse(0,0,clusters,data)


C:\Users\justi\AppData\Local\Temp\ipykernel_23148\2454418393.py:5: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda', nogil=True, cache=True)
C:\Users\justi\AppData\Local\Temp\ipykernel_23148\2454418393.py:5: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "sse" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_23148\2454418393.py (5)

File "..\..\..\..\AppData\Local\Temp\ipykernel_23148\2454418393.py", line 5:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda', nogil=True, cache=True)
C:\Users\justi\AppDat

44867.00341312835

In [5]:

# clusters: clusters df
# data: points df
# c: the cluster assignment array
@jit(target_backend='cuda') 
def reassignPoints(data, clusters, c):
    for i,ii in data.iterrows():
        bestIndex = -1
        bestSSE = 100000000
        for j,jj in clusters.iterrows():
            currentSSE = sse(i, j, clusters, data)
            if(currentSSE < bestSSE):
                bestSSE = currentSSE
                bestIndex = j
        c[i] = bestIndex

reassignPoints(data, clusters, c)

C:\Users\justi\AppData\Local\Temp\ipykernel_23148\68328681.py:4: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_23148\68328681.py:4: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "reassignPoints" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_23148\68328681.py (4)

File "..\..\..\..\AppData\Local\Temp\ipykernel_23148\68328681.py", line 4:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_23148\68328681.py:4: N

In [6]:
# clusters: clusters df
# data: points df
# c: the cluster assignment array
@jit(target_backend='cuda') 
def totalError(c, clusters, data):
    error = 0.0
    for i in range(len(c)):
        error = error + sse(i, c[i], clusters, data)
    return(error)

totalError(c, clusters, data)

C:\Users\justi\AppData\Local\Temp\ipykernel_23148\1850685268.py:4: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_23148\1850685268.py:4: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "totalError" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_23148\1850685268.py (4)

File "..\..\..\..\AppData\Local\Temp\ipykernel_23148\1850685268.py", line 4:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_23148\1850685268.p

1662359703.4717348

In [7]:
@jit(target_backend='cuda') 
def clusterContent(c, t):
    total = 0
    for i in range(len(c)):
        if (c[i] == t):
            total = total + 1
    return (total)

clusterContent(c, 0)


C:\Users\justi\AppData\Local\Temp\ipykernel_23148\845319178.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')


1

In [8]:
# clusters: clusters df
# data: points df
# c: the cluster assignment array
# cluster: the focal cluster
@jit(target_backend='cuda') 
def clusterError(c, cluster, clusters, data):
    error = 0.0
    for i in range(len(c)):
        if(c[i] == cluster):
            error = error + sse(i, c[i], clusters, data)
    return(error)

clusterError(c, 0, clusters, data)

C:\Users\justi\AppData\Local\Temp\ipykernel_23148\4154215476.py:5: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_23148\4154215476.py:5: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "clusterError" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_23148\4154215476.py (5)

File "..\..\..\..\AppData\Local\Temp\ipykernel_23148\4154215476.py", line 5:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_23148\4154215476

12907.730127774032

In [9]:
@jit(target_backend='cuda') 
def moveClusters(c, clusters, data):
    for i,ii in clusters.iterrows():
        for j,jj in enumerate(clusters.iloc[i]):
            sum = 0.0
            totP = 0.0
            for k in range(len(c)):
                if (c[k] == i):
                    sum = sum + data.iloc[k][j]
                    totP = totP + 1.0
            if (totP != 0):
                clusters.iloc[i][j] = sum/totP

print(clusters)
moveClusters(c, clusters, data)
print(clusters)

C:\Users\justi\AppData\Local\Temp\ipykernel_23148\2903512841.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_23148\2903512841.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "moveClusters" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_23148\2903512841.py (1)

File "..\..\..\..\AppData\Local\Temp\ipykernel_23148\2903512841.py", line 1:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_23148\2903512841

   Severity  Temperature(F)  Wind_Chill(F)  Humidity(%)  Pressure(in)  \
0       4.0       45.280928     -18.049335    26.531568     28.272819   
1       4.0       19.632528     114.092487    73.229098     28.589851   
2       4.0       13.549517     -34.412489     9.207838     29.914190   

   Visibility(mi)  Wind_Speed(mph)  Precipitation(in)  
0       33.097954       211.783523           3.097339  
1        1.664061       149.680531           1.437676  
2       76.809146       213.445194           1.148373  


c:\Users\justi\AppData\Local\Programs\Python\Python311\Lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "moveClusters" was compiled in object mode without forceobj=True.

File "..\..\..\..\AppData\Local\Temp\ipykernel_23148\2903512841.py", line 3:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
c:\Users\justi\AppData\Local\Programs\Python\Python311\Lib\site-packages\numba\core\object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected. This is deprecated behaviour that will be removed in Numba 0.59.0.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit

File "..\..\..\..\AppData\Local\Temp\ipykernel_23148\2903512841.py", line 3:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaDeprecationWarning(msg,


   Severity  Temperature(F)  Wind_Chill(F)  Humidity(%)  Pressure(in)  \
0       4.0       84.000000      84.000000    33.000000     29.190000   
1       4.0       58.217779      56.455001    67.646135     29.195434   
2       4.0       13.549517     -34.412489     9.207838     29.914190   

   Visibility(mi)  Wind_Speed(mph)  Precipitation(in)  
0       10.000000       232.000000           0.000000  
1        8.893629         7.428284           0.005555  
2       76.809146       213.445194           1.148373  


In [10]:
@jit(target_backend='cuda') 
def KMeans(data, k, eps, iterationsMax):
    clusters = pd.DataFrame(data = 0.0, columns=data.columns, index=range(k))
    c = np.full(len(data), -1)
    initClusters(data, clusters)

    oldError = -1 * eps
    newError = 0

    for a in range(1,iterationsMax):
        reassignPoints(data, clusters, c)
        moveClusters(c, clusters, data)
        newError = totalError(c, clusters, data)
        #print(newError)
        if(abs(newError - oldError) < eps):
            break
        oldError = newError
    #print(newError)
    return(newError)
#KMeans(data, k, eps, iterationsMax)

C:\Users\justi\AppData\Local\Temp\ipykernel_23148\1331116103.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')


In [11]:
@jit(target_backend='cuda') 
def KMeans2(data, k, eps, iterationsMax):
    clusters = pd.DataFrame(data = 0.0, columns=data.columns, index=range(k))
    c = np.full(len(data), -1)
    initClusters(data, clusters)

    oldError = -1 * eps
    newError = 0

    for a in range(1,iterationsMax):
        reassignPoints(data, clusters, c)
        moveClusters(c, clusters, data)
        newError = totalError(c, clusters, data)
        #print(newError)
        if(abs(newError - oldError) < eps):
            break
        oldError = newError
    #print(newError)
    output = clusters
    cle = np.full(k,-1.0)
    ccn = np.full(k, -1)
    for i in range(k):
        cle[i] = round(clusterError(c, i, clusters, data),3)
        ccn[i] = clusterContent(c, i)
    output['error'] = cle
    output['numOfPoints'] = ccn

    return(output)

start = timer()
result = KMeans2(data, k, eps, iterationsMax)
print(result)
print("without GPU:", timer()-start)   

C:\Users\justi\AppData\Local\Temp\ipykernel_23148\2065996003.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_23148\2065996003.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "KMeans2" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_23148\2065996003.py (1)

File "..\..\..\..\AppData\Local\Temp\ipykernel_23148\2065996003.py", line 1:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_23148\2065996003.py:1

   Severity  Temperature(F)  Wind_Chill(F)  Humidity(%)  Pressure(in)  \
0       4.0       84.000000      84.000000    33.000000     29.190000   
1       4.0       39.669438      35.495317    77.864284     29.125692   
2       4.0       71.603159      71.580522    60.272225     29.245763   

   Visibility(mi)  Wind_Speed(mph)  Precipitation(in)         error  \
0       10.000000       232.000000           0.000000  0.000000e+00   
1        7.959825         7.455448           0.007389  1.858755e+07   
2        9.567508         7.408681           0.004231  2.857046e+07   

   numOfPoints  
0            1  
1        26268  
2        36400  
without GPU: 3448.749397300184
